In [ ]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
import tkinter as tk
from tkinter import messagebox

# Read the data from CSV files
credits_df = pd.read_csv("credits.csv")
movies_df = pd.read_csv("movies.csv")

# Merge the dataframes
movies_df = movies_df.merge(credits_df, on='title')

# Select relevant columns
movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# Drop missing values
movies_df.dropna(inplace=True)

# Define helper functions
def convert(obj):
    return [i['name'] for i in ast.literal_eval(obj)]

def convert3(obj):
    return [i['name'] for i in ast.literal_eval(obj)][:3]

def fetch_director(obj):
    return [i['name'] for i in ast.literal_eval(obj) if i['job'] == 'Director']

def stem(text):
    ps = PorterStemmer()
    return " ".join([ps.stem(i) for i in text.split()])

# Apply data transformations
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)
movies_df['cast'] = movies_df['cast'].apply(convert3)
movies_df['crew'] = movies_df['crew'].apply(fetch_director)
movies_df['overview'] = movies_df['overview'].apply(lambda x: x.split())
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']

# Create a new DataFrame with selected columns
new_df = movies_df[['movie_id', 'title', 'tags']]

# Correcting the code to avoid SettingWithCopyWarning
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x) if isinstance(x, (list, tuple)) else str(x))
new_df['tags'] = new_df['tags'].str.lower()
new_df['tags'] = new_df['tags'].apply(stem)

# Create the count matrix
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

# Compute cosine similarity matrix
similarity = cosine_similarity(vectors)

def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    return [new_df.iloc[i[0]].title for i in movies_list]

def display_recommendations(movie, recommendations):
    messagebox.showinfo("Recommendations", f"Recommended movies for '{movie}':\n\n" + '\n'.join([f"{i+1}. {recommendation}" for i, recommendation in enumerate(recommendations)]))

def get_recommendations():
    movie = entry.get()
    if movie:
        recommendations = recommend(movie)
        display_recommendations(movie, recommendations)
    else:
        messagebox.showwarning("Empty Field", "Please enter a movie title.")

# GUI
root = tk.Tk()
root.title("Movie Recommendation System")

label = tk.Label(root, text="Enter a movie title:")
label.pack(pady=10)

entry = tk.Entry(root, font=('Arial', 14))
entry.pack(pady=10)

button = tk.Button(root, text="Get Recommendations", command=get_recommendations)
button.pack(pady=10)

root.mainloop()
